In [1]:
import json

import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!cp drive/MyDrive/libraries/books.jsn ./
!cp drive/MyDrive/libraries/dataset_knigi_1.xlsx ./

In [15]:
books = pd.read_json("books.jsn")
books["serial_name"] = books["serial_name"].apply(lambda x: x.lower() if x else "")
books["material_name"] = books["serial_name"].apply(lambda x: x.lower() if x else "")
books["rubric_name"] = books["serial_name"].apply(lambda x: x.lower() if x else "")
books["main"] = (books["author_fullName"].str.lower() +
                 " [SEP] " +
                 books["title"].apply(
                     lambda x: x.lower().split(":")[0].strip()
                 ))
# group by author + book name
books = (books.groupby(["main"])
    .agg(
        bbk=("bbk", "first"),
        title=("title", "first"),
        serial_name=("serial_name", "max"),
        material_name=("material_name", "max"),
        annotation=("annotation", "max"),
        rubric_name=("rubric_name", "max"),
        author_fullName=("author_fullName", "first"),
        id=(
            "id",
            lambda x: sorted(list(set(x))),
        ),
    )
    .reset_index()
)

books["text"] = (books["main"] + " [SEP] " +
                 books["rubric_name"].str.lower() + " [SEP] " +
                 books["serial_name"].str.lower() + " [SEP] " +
                 books["material_name"].str.lower() + " [SEP] " +
                 books["annotation"].str.lower())
books.to_csv("text_csv.csv", index=None)
books_dict = dict()
for i, row in books.iterrows():
    ids = row["id"]
    for book_id in ids:
        books_dict[book_id] = row["text"]
books

,main,bbk,title,serial_name,material_name,annotation,rubric_name,author_fullName,id,text
0,"clark mary higgins [SEP] weep no more, my lady",84,"Weep No More, My Lady",,,,,Clark Mary Higgins,[1177634],"clark mary higgins [SEP] weep no more, my lady..."
1,connelly michael [SEP] a darkness more than night,81.2Англ,A Darkness More Than Night : a novel; [кн. на ...,,,A Darkness More Than Night is the tenth novel ...,,Connelly Michael,[1627543],connelly michael [SEP] a darkness more than ni...
2,connelly michael [SEP] city of bones,81.2Англ,City of Bones : [кн. на англ. яз.],,,"On New Year's Day, Detective Harry Bosch field...",,Connelly Michael,[1627533],connelly michael [SEP] city of bones [SEP] [S...
3,connelly michael [SEP] echo park,84,Echo Park,,,,,Connelly Michael,[1178414],connelly michael [SEP] echo park [SEP] [SEP] ...
4,connelly michael [SEP] the drop,84,The Drop,,,,,Connelly Michael,[1028952],connelly michael [SEP] the drop [SEP] [SEP] ...
...,...,...,...,...,...,...,...,...,...,...
3833,ярышевская елена николаевна [SEP] вышел пудель...,Д,Вышел пудель погулять,строчка за строчкой с сыном и дочкой,строчка за строчкой с сыном и дочкой,"Вы знаете, какие собаки считаются самыми мален...",строчка за строчкой с сыном и дочкой,Ярышевская Елена Николаевна,[1951187],ярышевская елена николаевна [SEP] вышел пудель...
3834,ясницкий леонид нахимович [SEP] современные пр...,72,Современные проблемы науки : учебное пособие,,,Рассмотрены вопросы становления и эволюции нау...,,Ясницкий Леонид Нахимович,[1758200],ясницкий леонид нахимович [SEP] современные пр...
3835,яхина гузель шамилевна [SEP] дети мои,84(2Рос)6,Дети мои : роман,проза гузель яхиной,проза гузель яхиной,"Новый роман лауреата премий ""Большая книга"" и ...",проза гузель яхиной,Яхина Гузель Шамилевна,"[324421, 1512269]",яхина гузель шамилевна [SEP] дети мои [SEP] пр...
3836,яхина гузель шамилевна [SEP] зулейха открывает...,84(2Рос)6,Зулейха открывает глаза,проза гузель яхиной,проза гузель яхиной,"Гузель Яхина родилась и выросла в Казани, окон...",проза гузель яхиной,Яхина Гузель Шамилевна,"[1389555, 1814545]",яхина гузель шамилевна [SEP] зулейха открывает...


In [19]:
books["text"].values[0]

'clark mary higgins [SEP] weep no more, my lady [SEP]  [SEP]  [SEP]  [SEP] '

In [20]:
!iconv -f cp1251 < drive/MyDrive/libraries/datasets_biblioteki/datasets_2/siglas.csv | head

id;cbs;adress;name;eisk;;
1;3;ул. Клары Цеткин, д. 11, корп. 1;21-1;284375696001;;
2;3;ул. Космонавта Волкова, д. 3;21-2;284375229001;;
3;3;Ленинградское ш., д. 114;22-1;284384346001;;
4;3;Ленинградское ш., д. 120;22-2;284384353001;;
5;3;ул. Астрадамская, д. 4;23-1;284385857001;;
6;3;ул. Тимирязевская, д. 17, корп. 1;23-2;284385867001;;
7;3;ул. Новопесчаная, д. 23 корп.7;24;284385847001;;
8;3;ул. Врубеля, д. 13;25;284385826001;;
9;3;ул. Зеленоградская, д. 27А;26;284385821001;;


In [21]:
!iconv -f cp1251 < drive/MyDrive/libraries/datasets_biblioteki/datasets_2/cat_1.csv | head

recId;aut;title;place;publ;yea;lan;rubrics;person;serial;material;biblevel;ager
1;Устинова Татьяна Витальевна;Призрак Канта;Москва;Эксмо;2018;rus;Художественная литература : Российский детектив;;Татьяна Устинова. Первая среди лучших;Текст. материал;Монография;16+
2;;Наука и жизнь;Москва;Редакция журнала «Наука и жизнь»;1938-;rus;Наука. Науковедение;;;Текст. материал;Периодика;12+
3;;Наука и жизнь;;;2018;rus;Наука. Науковедение;;;Текст. материал;Монография;12+
4;;Наука и жизнь;;;2018;rus;Наука. Науковедение;;;Текст. материал;Монография;12+
5;;Наука и жизнь;;;2018;rus;Наука. Науковедение;;;Текст. материал;Монография;12+
6;Тараканов Борис Игоревич;Кольцо времени;Москва;АиФ Принт;2001;rus;Художественная литература : Фэнтэзи;;Черная звезда;Текст. материал;Монография;
8;;Фантастика-86;Москва;Молодая гвардия;1986;;Художественная литература;;;Текст. материал;Монография;
9;;Фантастика-90;Москва;Молодая гвардия;1990;;Художественная литература : Фантастика;;;Текст. материал;Монография;
10;Федотов

In [26]:
cat_2 = pd.read_csv("drive/MyDrive/libraries/"
                    "datasets_biblioteki/datasets_2/cat_2.csv",
                    encoding="cp1251",
                    sep=";"
)

In [29]:
cat_2[cat_2["recId"] == 1389555]

,recId,aut,title,place,publ,yea,lan,rubrics,person,serial,material,biblevel,ager
251569,1389555,Яхина Гузель Шамилевна,Зулейха открывает глаза,Москва,"АСТ , Редакция Елены Шубиной",2020,rus,Художественная литература : Проза : Российская...,NaN,Проза Гузель Яхиной,Текст. материал,Монография,12+


In [30]:
cat_2[cat_2["recId"] == 1814545]

,recId,aut,title,place,publ,yea,lan,rubrics,person,serial,material,biblevel,ager
519205,1814545,Яхина Гузель Шамилевна,Зулейха открывает глаза,Москва,"Редакция Елены Шубиной , АСТ",2016,rus,Художественная литература,NaN,NaN,Текст. материал,Монография,NaN


In [40]:
interactions = pd.read_excel("drive/MyDrive/libraries/dataset_knigi_1.xlsx")
interactions["book_id"] = interactions["source_url"].str.split("/").str[-2].astype(int)
interactions["book_str"] = interactions["book_id"].replace(books_dict)
print(interactions["event"].unique())
interactions

['add' 'create_order']


,source_url,event,dt,user_id,book_id,book_str
0,https://www.mos.ru/knigi/book/1163786/,add,2021-06-01,1,1163786,бэйли гвен [SEP] о чем думает ваша собака? [SE...
1,https://www.mos.ru/knigi/book/66612/,add,2021-06-01,1,66612,яковлев юрий яковлевич [SEP] у человека должна...
2,https://www.mos.ru/knigi/book/1034596/,add,2021-06-01,1,1034596,"баранова марта петровна [SEP] тяпа, борька и р..."
3,https://www.mos.ru/knigi/book/1152760/,add,2021-06-01,1,1152760,"дункан дэвид дуглас [SEP] лумп - собака, котор..."
4,https://www.mos.ru/knigi/book/16168/,add,2021-06-01,1,16168,брюгген мария ван [SEP] почему собака лает и е...
...,...,...,...,...,...,...
4181,https://www.mos.ru/knigi/book/1024959/,create_order,2021-03-30,103,1024959,носов николай [SEP] незнайка на луне [SEP] [S...
4182,https://www.mos.ru/knigi/book/915148/,create_order,2021-03-30,103,915148,поликарпова татьяна николаевна [SEP] листья бу...
4183,https://www.mos.ru/knigi/book/1859007/,create_order,2021-03-30,103,1859007,лабковский михаил александрович [SEP] про любо...
4184,https://www.mos.ru/knigi/book/218302/,create_order,2021-03-31,103,218302,"андерсон пол уильям [SEP] операция ""хаос""; три..."


In [102]:
# interactions[interactions["event"] == "create_order"]
mos_user_book_dict = (
    interactions.groupby("user_id")
                .apply(lambda x: set(
                                 list(x["book_id"])
                                 )
                )
                .to_dict()
                )
str(list(mos_user_book_dict.items())[1])

'(2, {1652993, 1323524, 12556, 1653518, 2522648, 1600025, 2113053, 1445920, 2111270, 1588519, 1650986, 1652011, 1055787, 2005551, 266291, 265277, 1594174, 265283, 2510920, 1610570, 871755, 2109774, 1954135, 2005595, 961371, 267615, 1210981, 2522473, 12651, 2119021, 1957742, 1962235, 1847411, 1818996, 2496376, 2496123, 1721723, 1652350, 1421694, 1224064, 1648003, 2097031, 1863048, 6135, 735129, 2088604, 1653149, 2024605, 1429919, 1801641, 1569973, 871094, 2111674, 1595834, 352956, 1863356, 274367, 2111686, 1554631, 1554632, 1554637, 1554638, 2107090, 1863382, 1238, 1565662, 1853920, 2522594, 1001703, 6128, 906231, 2111224, 42235})'

In [51]:
!iconv -f cp1251 < drive/MyDrive/libraries/datasets_biblioteki/datasets_2/circulaton_1.csv | head

circulationID;catalogueRecordID;barcode;startDate;finishDate;readerID;bookpointID;state;;
2950477;12555;0381002128634;01.09.2019;01.09.2019;1188;46;Размещен в фонде;;
2950479;841695;1086000549854;01.09.2019;01.09.2019;23276;56;Размещен в фонде;;
2950480;553535;1085000292739;01.09.2019;15.10.2019;23128;55;Размещен в фонде;;
2950481;272495;1085000292722;01.09.2019;15.10.2019;23128;55;Размещен в фонде;;
2950482;103580;0181001063192;01.09.2019;01.10.2019;44894;99;Переоформлен;;
2950483;244035;0181005267466;01.09.2019;01.10.2019;44894;99;Переоформлен;;
2950484;11847;0181005267701;01.09.2019;01.10.2019;44894;99;Переоформлен;;
2950485;460449;0181005267381;01.09.2019;01.10.2019;44894;99;Переоформлен;;
2950486;551631;0181005267510;01.09.2019;01.10.2019;44894;99;Переоформлен;;


In [116]:
from glob import glob
circulaton_files = sorted(glob("drive/MyDrive/libraries/datasets_biblioteki/datasets_2/circulaton_*.csv"))
# print(circulaton_files[:5])
circulaton_df = None
print("-" * 80 )
for filename in circulaton_files:
    print(f"\r{filename.ljust(5)}", end="")
    temp_df = pd.read_csv(filename, encoding="cp1251", sep=";")
    temp_df = temp_df[["catalogueRecordID", "readerID", "startDate",
                       "finishDate", "bookpointID", "state"]]
    if circulaton_df is None:
        circulaton_df = temp_df
    else:
        circulaton_df = pd.concat([circulaton_df, temp_df])
circulaton_df

--------------------------------------------------------------------------------
drive/MyDrive/libraries/datasets_biblioteki/datasets_2/circulaton_9.csv

,catalogueRecordID,readerID,startDate,finishDate,bookpointID,state
0,12555,1188,01.09.2019,01.09.2019,46,Размещен в фонде
1,841695,23276,01.09.2019,01.09.2019,56,Размещен в фонде
2,553535,23128,01.09.2019,15.10.2019,55,Размещен в фонде
3,272495,23128,01.09.2019,15.10.2019,55,Размещен в фонде
4,103580,44894,01.09.2019,01.10.2019,99,Переоформлен
...,...,...,...,...,...,...
999995,1593756,508023,02.02.2021,02.02.2021,296,Размещен в фонде
999996,1663896,508023,02.02.2021,02.02.2021,296,Размещен в фонде
999997,1607447,508023,02.02.2021,02.02.2021,296,Размещен в фонде
999998,1576132,508023,02.02.2021,02.02.2021,296,Размещен в фонде


In [83]:
circulation_reader_book_dict = (
    circulaton_df.groupby("readerID")
                 .apply(lambda x: set(list(x["catalogueRecordID"])))
                 .to_dict()
)

In [103]:
# матчи пользователей из циркулатона
# с пользователями мос.ру по айдишникам книжек
mos_circ_user = dict()
for user, user_books in mos_user_book_dict.items():
    max_intersect = 0
    max_circ_user = None
    circ_books_total = 0
    for circ_reader, circ_books in circulation_reader_book_dict.items():
        intersection = len(user_books.intersection(circ_books))
        if intersection > max_intersect:
            max_intersect = intersection
            max_circ_user = circ_reader
            circ_books_total = len(circ_books)
    if max_intersect > 0:
        mos_circ_user[user] = {"circ_user": max_circ_user,
                            "intersection": max_intersect,
                            "circ_books_total": circ_books_total,
                            "mos_perc": max_intersect / len(user_books),
                            "circ_perc": max_intersect / circ_books_total,
                            "mos_books": len(user_books)
                            }
        print(f"\r{user}: {str(mos_circ_user[user])}", end="")
mos_circ_user_df = pd.DataFrame(mos_circ_user).T
mos_circ_user_df

103: {'circ_user': 2484, 'intersection': 3, 'circ_books_total': 427, 'mos_perc': 0.046153846153846156, 'circ_perc': 0.00702576112412178, 'mos_books': 65}

,circ_user,intersection,circ_books_total,mos_perc,circ_perc,mos_books
1,647497.0,16.0,25.0,0.120301,0.640000,133.0
2,6313.0,7.0,628.0,0.095890,0.011146,73.0
3,2612.0,1.0,13.0,0.071429,0.076923,14.0
4,566218.0,5.0,37.0,0.384615,0.135135,13.0
5,636792.0,17.0,59.0,0.202381,0.288136,84.0
...,...,...,...,...,...,...
99,4969.0,1.0,318.0,0.090909,0.003145,11.0
100,380048.0,5.0,42.0,0.263158,0.119048,19.0
101,219058.0,13.0,93.0,0.342105,0.139785,38.0
102,570476.0,11.0,83.0,0.500000,0.132530,22.0


In [113]:
circulaton_df

,catalogueRecordID,readerID
0,12555,1188
1,841695,23276
2,553535,23128
3,272495,23128
4,103580,44894
...,...,...
999995,1593756,508023
999996,1663896,508023
999997,1607447,508023
999998,1576132,508023


In [130]:
# полностью совпавший юзер
print(mos_circ_user_df[mos_circ_user_df["mos_perc"] == 1])
# mos interactions
print(interactions[interactions["user_id"] == 98][["book_id", "event", "book_str"]])
# circ interactions
print(circulaton_df[circulaton_df["readerID"] == 648840])
# circ interactions
user_98_books = set(
    interactions[interactions["user_id"] == 98]["book_id"].values)
print(circulaton_df[
                    (circulaton_df["catalogueRecordID"].
                     apply(lambda x: x in user_98_books)
                     ) &
                    (circulaton_df["readerID"] == 648840)
                    ])

    circ_user  intersection  circ_books_total  mos_perc  circ_perc  mos_books
98   648840.0           5.0              21.0       1.0   0.238095        5.0
      book_id         event                                           book_str
4026    95519  create_order  одленд николас [SEP] бобер, который наломал др...
4027  1554627  create_order  дюбюк марианна [SEP] почтальон мышка [SEP] поч...
4028    32392  create_order  буйе клер [SEP] что у волка на обед? [SEP] кни...
4029  1554655  create_order  рервик бьерн [SEP] кафе "птичий хвост" [SEP] д...
4030  1554247  create_order  карпова наталья владимировна [SEP] как хорошо ...
        catalogueRecordID  readerID  ... bookpointID             state
712390            1825520    648840  ...          28  Размещен в фонде
712393            2015056    648840  ...          28  Размещен в фонде
712400             318351    648840  ...          28  Размещен в фонде
712407            2015317    648840  ...          28  Размещен в фонде
712420         

по этому единственному пользователю
насколько я понимаю из circulaton 
нас интересует только статус "На руках",
но книжек в циркулатоне даже для этого пользователя
больше